In [1]:
# Install required libraries if not already installed
%pip install tensorflow keras opencv-python scikit-learn matplotlib
# Cross validation 1o folds
# Preprocessing
# Features extractoom and visulatisation
# Svm and hog
# Change in layers and write everytbing
# Onr more dataset
# The research paper has the training accuracy of 96.47% and validation accuracy of 93.44%
# Our implementation has training accuracy of 97.67% and validation accuracy of 92.97%


Note: you may need to restart the kernel to use updated packages.


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


In [14]:
# Set up the paths to the training and testing data
train_path = 'Training'
test_path = 'Testing'

# Define image size and batch size
img_height, img_width = 150, 150
batch_size = 32


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data preprocessing for testing
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Assuming multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # Assuming multi-class classification
)


Found 2870 images belonging to 4 classes.
Found 394 images belonging to 4 classes.


In [16]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Load the VGG16 model, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Define the new model
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Summary of the model
model.summary()


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ ?                      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Define training parameters
epochs = 10  # You can adjust the number of epochs as needed
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = test_generator.samples // test_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=validation_steps
)


Epoch 1/10


C:\Users\techn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


89/89 ━━━━━━━━━━━━━━━━━━━━ 819s 9s/step - accuracy: 0.3584 - loss: 1.3826 - val_accuracy: 0.2917 - val_loss: 1.8175
Epoch 2/10
 1/89 ━━━━━━━━━━━━━━━━━━━━ 15:20 10s/step - accuracy: 0.5312 - loss: 1.0284

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


89/89 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.5312 - loss: 0.5200 - val_accuracy: 0.2000 - val_loss: 0.8349
Epoch 3/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 737s 8s/step - accuracy: 0.6354 - loss: 0.9206 - val_accuracy: 0.4219 - val_loss: 1.8700
Epoch 4/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.8438 - loss: 0.2304 - val_accuracy: 0.5000 - val_loss: 0.8909
Epoch 5/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 722s 8s/step - accuracy: 0.7830 - loss: 0.5770 - val_accuracy: 0.6849 - val_loss: 1.6112
Epoch 6/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9375 - loss: 0.1498 - val_accuracy: 0.6000 - val_loss: 1.2151
Epoch 7/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 734s 8s/step - accuracy: 0.8722 - loss: 0.3787 - val_accuracy: 0.5833 - val_loss: 1.6846
Epoch 8/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8438 - loss: 0.2087 - val_accuracy: 0.7000 - val_loss: 0.5850
Epoch 9/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 720s 8s/step - accuracy: 0.8695 - loss: 0.3560 - val_accuracy: 0.6797 - val_loss: 1.509

In [19]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}")



13/13 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.6724 - loss: 1.5040
Test Accuracy: 0.6599
Test Loss: 1.6507


In [23]:
# Perform a dummy prediction to initialize the model's input
_ = model.predict(train_generator, steps=1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
